#### Testing CuPy for the first time

In [1]:
# Import necessary libraries
import cupy as cp
import numpy as np
import time

In [2]:
!pwd

/home/rapids/notebooks


In [3]:
!nvidia-smi

Thu Sep 18 07:07:00 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090        Off | 00000000:09:00.0 Off |                  N/A |
| 36%   39C    P8              35W / 370W |     26MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
cp.show_config()

OS                           : Linux-5.15.0-58-generic-x86_64-with-glibc2.35
Python Version               : 3.11.13
CuPy Version                 : 13.6.0
CuPy Platform                : NVIDIA CUDA
NumPy Version                : 2.2.6
SciPy Version                : 1.16.2
Cython Build Version         : 3.1.3
Cython Runtime Version       : None
CUDA Root                    : /opt/conda
nvcc PATH                    : /opt/conda/bin/nvcc
CUDA Build Version           : 12090
CUDA Driver Version          : 12030
CUDA Runtime Version         : 12090 (linked to CuPy) / 12000 (locally installed)
CUDA Extra Include Dirs      : []
cuBLAS Version               : (available)
cuFFT Version                : 11000
cuRAND Version               : 10301
cuSOLVER Version             : (11, 4, 2)
cuSPARSE Version             : (available)
NVRTC Version                : (12, 0)
Thrust Version               : 200802
CUB Build Version            : 200800
Jitify Build Version         : <unknown>
cuDNN Build Ve

#### On the **CPU** with **NumPy**

In [5]:
# --- On the CPU with NumPy ---
start_time_cpu = time.time()
# Create a large array on the CPU
cpu_array = np.random.rand(10000, 10000)
# Perform a matrix multiplication on the CPU
result_cpu = np.dot(cpu_array, cpu_array)
end_time_cpu = time.time()
print(f"NumPy execution time: {end_time_cpu - start_time_cpu:.4f} seconds")

NumPy execution time: 4.9810 seconds


#### On the **GPU** with **CuPy**

In [6]:
# --- On the GPU with CuPy ---
start_time_gpu = time.time()
# Now, copy the *exact same* array to the GPU
gpu_array = cp.asarray(cpu_array)

# Perform a matrix multiplication on the GPU
result_gpu = cp.dot(gpu_array, gpu_array)
end_time_gpu = time.time()
print(f"CuPy execution time: {end_time_gpu - start_time_gpu:.4f} seconds")

CuPy execution time: 0.3299 seconds


In [7]:
# --- Verify the result ---
# To inspect the result on the CPU, transfer it from the GPU
result_gpu_host = cp.asnumpy(result_gpu)
print("\nResults match:", np.allclose(result_cpu, result_gpu_host))


Results match: True


#### You can use `xp` to handle both cases.
```python
import numpy as np

try:
    import cupy as cp
    # 기본 GPU (장치 0번) 초기화를 시도
    with cp.cuda.Device(0):
        xp = cp
        print("Using CuPy. GPU device 0 is active.")
except (ImportError, cp.cuda.runtime.CUDARuntimeError):
    # CuPy가 없거나, 장치 초기화에 실패하면 NumPy로 전환
    xp = np
    print("Using NumPy. CuPy not installed or device initialization failed.")
```